In [ ]:
file_path = "combined_dataset_clean.csv"
with open(file_path, "r", encoding="utf-8") as f:
    raw_content = f.readlines()

header_line = ""
for line in raw_content:
    if line.strip().startswith("http"):
        continue
    if line.strip():
        header_line = line.strip()
        break

merged_lines = []
header_found = False
for line in raw_content:
    if not header_found and line.strip() == header_line:
        header_found = True
        continue  
    if line.strip().startswith("http"):
        merged_lines.append(line.strip())
    else:
        if merged_lines:
            merged_lines[-1] += " " + line.strip()

header = next(csv.reader([header_line], quotechar='"', skipinitialspace=True))
reader = csv.reader(merged_lines, quotechar='"', skipinitialspace=True)
rows = [row for row in reader if any(cell.strip() for cell in row)]

max_len = max(len(row) for row in rows)
normalized_rows = [row + [''] * (max_len - len(row)) for row in rows]
header += [''] * (max_len - len(header))  

df = pd.DataFrame(normalized_rows, columns=header)
df.to_csv("FINAL_COMBINED.csv", index=False)


In [7]:
print("Total rows:", df.shape[0])

Total rows: 332


## MAPPING

In [ ]:
import pandas as pd


disease_df = pd.read_csv("Disease.csv")
food_df = pd.read_csv("FINAL_COMBINED.csv")


food_df.columns = [col.strip().lower().replace(" ", "_") for col in food_df.columns]


disorder_keywords = {
    "Inherited musculoskeletal disorders": [
        "joint", "mobility", "hip", "glucosamine", "arthritis", "cartilage", "flexibility", "movement"
    ],
    "Inherited gastrointestinal disorders": [
        "digest", "stomach", "bowel", "gastro", "sensitive", "flatulence", "diarrhea", "gut", "ibs", "digestive tract"
    ],
    "Inherited endocrine disorders": [
        "thyroid", "metabolism", "weight", "diabetes", "insulin", "hormone", "obesity", "hypothyroid", "glucose"
    ],
    "Inherited eye disorders": [
        "vision", "eye", "retina", "cataract", "antioxidant", "sight", "vision health", "ocular", "macula"
    ],
    "Inherited nervous system disorders": [
        "brain", "seizure", "cognitive", "nerve", "neuro", "neurological", "nerve support", "cognition", "focus"
    ],
    "Inherited cardiovascular disorders": [
        "heart", "cardiac", "circulation", "omega-3", "blood pressure", "circulatory", "cardio", "vascular"
    ],
    "Inherited skin disorders": [
        "skin", "derma", "allergy", "itch", "coat", "omega-6", "dermatitis", "eczema", "dry skin", "flaky", "skin barrier"
    ],
    "Inherited immune disorders": [
        "immune", "defense", "resistance", "inflammatory", "allergy defense", "autoimmune", "resistance", "defend"
    ],
    "Inherited disorders of the urinary and reproductive systems": [
        "urinary", "bladder", "kidney", "renal", "reproductive", "incontinence", "urine", "prostate", "neutered", "urination"
    ],
    "Inherited respiratory disorders": [
        "breath", "respiratory", "airway", "lung", "cough", "snoring", "breathing", "bronchial", "nasal"
    ],
    "Inherited blood disorders": [
        "anemia", "blood", "iron", "hemoglobin", "platelets", "clotting", "hemophilia", "blood sugar"
    ]
}



for col in ["product_title", "key_benefits", "need/preference", "alternate_product_recommendation"]:
    food_df[col] = food_df[col].astype(str).str.lower()

seen_products = set()
deduplicated_matches = []

for disorder, keywords in disorder_keywords.items():
    for _, row in food_df.iterrows():
        product_id = row["product_title"]

        if (disorder, product_id) in seen_products:
            continue

        for field in ["product_title", "key_benefits", "need/preference", "alternate_product_recommendation"]:
            if any(keyword in row[field] for keyword in keywords):
                deduplicated_matches.append({
                    "Disorder": disorder,
                    "Product Title": row["product_title"]
                })
                seen_products.add((disorder, product_id))
                break  


dedup_matches_df = pd.DataFrame(deduplicated_matches)


breed_disorder_df = disease_df[["Breed", "Disorder"]].drop_duplicates()
breed_food_matches_df = pd.merge(breed_disorder_df, dedup_matches_df, on="Disorder", how="inner").drop_duplicates()


In [24]:
breed_food_matches_df.to_csv("match1.csv", index=False)

In [ ]:
import pandas as pd

disease_df = pd.read_csv("Disease.csv")
food_df = pd.read_csv("FINAL_COMBINED.csv")

food_df.columns = [col.strip().lower().replace(" ", "_") for col in food_df.columns]

for col in ["product_title", "key_benefits", "need/preference", "alternate_product_recommendation"]:
    food_df[col] = food_df[col].astype(str).str.lower()

disorder_keywords = {
    "Inherited musculoskeletal disorders": [
        "joint", "mobility", "hip", "glucosamine", "arthritis", "cartilage", "flexibility", "movement"
    ],
    "Inherited gastrointestinal disorders": [
        "digest", "stomach", "bowel", "gastro", "sensitive", "flatulence", "diarrhea", "gut", "ibs", "digestive tract"
    ],
    "Inherited endocrine disorders": [
        "thyroid", "metabolism", "weight", "diabetes", "insulin", "hormone", "obesity", "hypothyroid", "glucose"
    ],
    "Inherited eye disorders": [
        "vision", "eye", "retina", "cataract", "antioxidant", "sight", "vision health", "ocular", "macula"
    ],
    "Inherited nervous system disorders": [
        "brain", "seizure", "cognitive", "nerve", "neuro", "neurological", "nerve support", "cognition", "focus"
    ],
    "Inherited cardiovascular disorders": [
        "heart", "cardiac", "circulation", "omega-3", "blood pressure", "circulatory", "cardio", "vascular"
    ],
    "Inherited skin disorders": [
        "skin", "derma", "allergy", "itch", "coat", "omega-6", "dermatitis", "eczema", "dry skin", "flaky", "skin barrier"
    ],
    "Inherited immune disorders": [
        "immune", "defense", "resistance", "inflammatory", "allergy defense", "autoimmune", "defend"
    ],
    "Inherited disorders of the urinary and reproductive systems": [
        "urinary", "bladder", "kidney", "renal", "reproductive", "incontinence", "urine", "prostate", "neutered", "urination"
    ],
    "Inherited respiratory disorders": [
        "breath", "respiratory", "airway", "lung", "cough", "snoring", "breathing", "bronchial", "nasal"
    ],
    "Inherited blood disorders": [
        "anemia", "blood", "iron", "hemoglobin", "platelets", "clotting", "hemophilia", "blood sugar"
    ]
}

recommendations = []


for _, row in disease_df[["Breed", "Disorder", "Disease"]].drop_duplicates().iterrows():
    breed = row["Breed"]
    disorder = row["Disorder"]
    disease = row["Disease"]
    keywords = disorder_keywords.get(disorder, [])
    
    # Поиск подходящих кормов по ключевым словам
    matched = []
    for _, food in food_df.iterrows():
        text_blob = " ".join([
            food["product_title"],
            food["key_benefits"],
            food["need/preference"],
            food["alternate_product_recommendation"]
        ])
        if any(keyword in text_blob for keyword in keywords):
            matched.append(food["product_title"])
        if len(matched) >= 3:  # до топ-3 рекомендаций
            break

    if matched:
        recommendations.append({
            "Breed": breed,
            "Disease": disease,
            "Disorder": disorder,
            "Recommended Recipes": ", ".join(matched)
        })


recommendation_df = pd.DataFrame(recommendations)


In [26]:
recommendation_df.to_csv("recommendation.csv", index=False)

## Content-based

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

disease_df = pd.read_csv("Disease.csv")
food_df = pd.read_csv("FINAL_COMBINED.csv")


food_df.columns = [col.strip().lower().replace(" ", "_") for col in food_df.columns]

# Перечень пищевых компонентов, которые мы добавим к описанию рецепта
nutrition_cols = [
    "moisture", "protein", "fat", "carbohydrate_(nfe)", "crude_fibre", "crude_ash", "soluble_fibre",
    "calcium", "phospohorus", "potassium", "sodium", "magnesium", "taurine", "vitamin_a", "vitamin_c",
    "vitamin_d", "vitamin_e", "thiamine_(vitamin_b1)", "riboflavin_(vitamin_b2)", "niacin_(vitamin_b3)",
    "pyridoxine_(vitamin_b6)", "pantothenic_acid_(vitamin_b5)", "folic_acid_(vitamin_b9)",
    "vitamin_b12_(cobalamin)", "biotin_(vitamin_b7)", "dha", "epa", "epa_+_dha", "omega-3-fatty_acids",
    "omega-6-fatty_acids", "beta-carotene", "l-carnitine", "glucosamine", "chondroitin_sulfate",
    "total_dietary_fibre", "iron", "copper", "zinc", "l-arginine", "glutamine_+_glutamate",
    "linoleic_acid", "essential_fatty_acids", "chloride", "sulphur", "hydroxyproline", "l-lysine",
    "dl-methionine_+_l-cystine", "insoluble_fibre", "starch", "bcaa_total", "total_dietary_fiber",
    "vitamin_k", "alpha-linolenic_acid_(ala)"
]


def create_recipe_text(row):
    parts = [
        row.get("product_title", ""),
        row.get("key_benefits", ""),
        row.get("ingredients", ""),
        row.get("need/preference", ""),
        row.get("food_form", ""),
        row.get("alternate_product_recommendation", ""),
        row.get("flavour", "")
    ]
    # Добавляем нутриенты в текст
    parts += [f"{col}_{row.get(col, '')}" for col in nutrition_cols if pd.notna(row.get(col))]
    return " ".join(str(p).lower() for p in parts if pd.notna(p))


food_df["recipe_text"] = food_df.apply(create_recipe_text, axis=1)


disease_df["problem_text"] = disease_df["Disorder"].str.lower() + " " + disease_df["Disease"].str.lower()

#  Векторизация с TF-IDF 
vectorizer = TfidfVectorizer(stop_words="english")

# Объединяем тексты заболеваний и рецептов
all_texts = pd.concat([disease_df["problem_text"], food_df["recipe_text"]], ignore_index=True)
tfidf_matrix = vectorizer.fit_transform(all_texts)

# Отдельно матрицы для болезней и рецептов
n_disease = disease_df.shape[0]
tfidf_disease = tfidf_matrix[:n_disease]
tfidf_recipes = tfidf_matrix[n_disease:]

#  Вычисление косинусного сходства 
similarity_matrix = cosine_similarity(tfidf_disease, tfidf_recipes)

#  Подбор топ-рецептов для каждой болезни 
top_k = 3
recommendations = []

for i, row in disease_df[["Breed", "Disease", "Disorder"]].iterrows():
    similar_indices = similarity_matrix[i].argsort()[::-1][:top_k]
    recommended_titles = [food_df.iloc[idx]["product_title"] for idx in similar_indices]
    recommendations.append({
        "Breed": row["Breed"],
        "Disease": row["Disease"],
        "Disorder": row["Disorder"],
        "Recommended Recipes": ", ".join(recommended_titles)
    })


recommendation_ml_df = pd.DataFrame(recommendations)



In [28]:
recommendation_ml_df.to_csv("reciepes.csv", index=False)

In [34]:
recommendation_ml_df

,Breed,Disease,Disorder,Recommended Recipes
0,affenpinscher,Cataracts,Inherited eye disorders,"Puppy No Corn, Wheat, Soy Chicken & Brown Rice..."
1,affenpinscher,Hip dysplasia,Inherited musculoskeletal disorders,Adult Healthy Mobility Large Breed Chicken Mea...
2,afghan hound,Hypothyroidism,Inherited endocrine disorders,"w/d Dry Dog Food with Chicken, w/d Wet Dog Foo..."
3,afghan hound,Cataracts,Inherited eye disorders,"Puppy No Corn, Wheat, Soy Chicken & Brown Rice..."
4,afghan hound,Retinal dysplasia,Inherited eye disorders,"Puppy No Corn, Wheat, Soy Chicken & Brown Rice..."
...,...,...,...,...
1242,yorkshire terrier,Shaker dog syndrome,Inherited nervous system disorders,"k/d starter kit Dog Food, Healthy Weight Dog T..."
1243,yorkshire terrier,tracheal collapse,Inherited respiratory disorders,"c/d Multicare Chicken Flavor Dog Food, Adult W..."
1244,yorkshire terrier,colour dilution alopecia,Inherited skin disorders,Adult Sensitive Stomach & Skin Tender Chicken ...
1245,yorkshire terrier,congenital hypotrichosis,Inherited skin disorders,Adult Sensitive Stomach & Skin Tender Chicken ...


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


disease_df = pd.read_csv("Disease.csv")
food_df = pd.read_csv("FINAL_COMBINED.csv")

food_df.columns = [col.strip().lower().replace(" ", "_") for col in food_df.columns]


# Питательные элементы
nutrition_cols = [
    "moisture", "protein", "fat", "carbohydrate_(nfe)", "crude_fibre", "crude_ash", "soluble_fibre",
    "calcium", "phospohorus", "potassium", "sodium", "magnesium", "taurine", "vitamin_a", "vitamin_c",
    "vitamin_d", "vitamin_e", "thiamine_(vitamin_b1)", "riboflavin_(vitamin_b2)", "niacin_(vitamin_b3)",
    "pyridoxine_(vitamin_b6)", "pantothenic_acid_(vitamin_b5)", "folic_acid_(vitamin_b9)",
    "vitamin_b12_(cobalamin)", "biotin_(vitamin_b7)", "dha", "epa", "epa_+_dha", "omega-3-fatty_acids",
    "omega-6-fatty_acids", "beta-carotene", "l-carnitine", "glucosamine", "chondroitin_sulfate",
    "total_dietary_fibre", "iron", "copper", "zinc", "l-arginine", "glutamine_+_glutamate",
    "linoleic_acid", "essential_fatty_acids", "chloride", "sulphur", "hydroxyproline", "l-lysine",
    "dl-methionine_+_l-cystine", "insoluble_fibre", "starch", "bcaa_total", "total_dietary_fiber",
    "vitamin_k", "alpha-linolenic_acid_(ala)"
]

#  текст рецепта
def create_recipe_text(row):
    parts = [
        row.get("product_title", ""),
        row.get("key_benefits", ""),
        row.get("ingredients", ""),
        row.get("need/preference", ""),
        row.get("food_form", ""),
        row.get("alternate_product_recommendation", ""),
        row.get("flavour", "")
    ]
    parts += [f"{col}_{row.get(col, '')}" for col in nutrition_cols if pd.notna(row.get(col))]
    return " ".join(str(p).lower() for p in parts if pd.notna(p))


food_df["recipe_text"] = food_df.apply(create_recipe_text, axis=1)


disease_df["problem_text"] = disease_df["Disorder"].str.lower() + " " + disease_df["*/"].str.lower()

#  Векторизация TF-IDF 
vectorizer = TfidfVectorizer(stop_words="english")
all_texts = pd.concat([disease_df["problem_text"], food_df["recipe_text"]], ignore_index=True)
tfidf_matrix = vectorizer.fit_transform(all_texts)

# Разделим обратно на болезни и рецепты
n_disease = disease_df.shape[0]
tfidf_disease = tfidf_matrix[:n_disease]
tfidf_recipes = tfidf_matrix[n_disease:]

# Косинусное сходство 
similarity_matrix = cosine_similarity(tfidf_disease, tfidf_recipes)

detailed_recommendations = []
top_k = 3

# Поля, которые включаем в результат
detail_fields = ["product_title", "key_benefits", "ingredients", "need/preference", "food_form"]
nutrient_fields = [
    "protein", "fat", "carbohydrate_(nfe)", "crude_fibre", "taurine", "vitamin_a",
    "vitamin_c", "vitamin_d", "vitamin_e", "omega-3-fatty_acids", "omega-6-fatty_acids",
    "l-carnitine", "glucosamine", "chondroitin_sulfate"
]

# Подбор рецептов
for i, row in disease_df[["Breed", "Disease", "Disorder"]].iterrows():
    similar_indices = similarity_matrix[i].argsort()[::-1][:top_k]
    for idx in similar_indices:
        food_row = food_df.iloc[idx]
        entry = {
            "Breed": row["Breed"],
            "Disease": row["Disease"],
            "Disorder": row["Disorder"],
        }
        for field in detail_fields:
            entry[field.replace("_", " ").title()] = food_row.get(field, "")
        for nut in nutrient_fields:
            entry[nut.replace("_", " ").title()] = food_row.get(nut, "")
        detailed_recommendations.append(entry)


detailed_df = pd.DataFrame(detailed_recommendations)


In [30]:
detailed_df.to_csv("detailed.csv", index=False)

In [32]:
detailed_df

,Breed,Disease,Disorder,Product Title,Key Benefits,Ingredients,Need/Preference,Food Form,Protein,Fat,...,Taurine,Vitamin A,Vitamin C,Vitamin D,Vitamin E,Omega-3-Fatty Acids,Omega-6-Fatty Acids,L-Carnitine,Glucosamine,Chondroitin Sulfate
0,affenpinscher,Cataracts,Inherited eye disorders,"Puppy No Corn, Wheat, Soy Chicken & Brown Rice...","Hill’s Science Diet Puppy No Corn, Wheat, Soy ...","Chicken, Brown Rice, Whole Grain Oats, Cracked...",NaN,Dry Food,29 %,20.2 %,...,NaN,NaN,185 ppm,NaN,740 IU/kg,1.31 %,4.05 %,NaN,NaN,NaN
1,affenpinscher,Cataracts,Inherited eye disorders,Puppy Large Breed Dry Dog Food with Chicken,Hill's Science Plan Large Breed Puppy food wit...,"COMPOSITION: Maize, chicken and turkey meal (c...",NaN,Dry Food,28.8 %,14.6 %,...,NaN,9664 IU/kg,149 ppm,718 IU/kg,650 IU/kg,0.93 %,2.76 %,334.7 ppm,NaN,NaN
2,affenpinscher,Cataracts,Inherited eye disorders,Puppy Medium Breed Dry Dog Food with Lamb & Rice,Hill's Science Plan Medium Puppy dog food with...,"COMPOSITION: Broken rice (27%), lamb meal (16%...",NaN,Dry Food,27.5 %,18.9 %,...,NaN,7984 IU/kg,112 ppm,688 IU/kg,580 IU/kg,1.01 %,2.61 %,NaN,NaN,NaN
3,affenpinscher,Hip dysplasia,Inherited musculoskeletal disorders,Adult Healthy Mobility Large Breed Chicken Mea...,Hill's Science Diet Healthy Mobility Large Bre...,"Chicken Meal, Brewers Rice, Whole Grain Sorghu...",NaN,Dry Food,22.4 %,14.7 %,...,NaN,NaN,202 ppm,NaN,708 IU/kg,1.29 %,3.77 %,NaN,644 ppm,NaN
4,affenpinscher,Hip dysplasia,Inherited musculoskeletal disorders,Adult Healthy Mobility Small Bites Chicken Mea...,Hill's Science Diet Healthy Mobility Small Bit...,"Chicken Meal, Brewers Rice, Whole Grain Sorghu...",NaN,Dry Food,22.4 %,14.7 %,...,NaN,NaN,202 ppm,NaN,708 IU/kg,1.29 %,3.77 %,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3736,yorkshire terrier,congenital hypotrichosis,Inherited skin disorders,Adult Sensitive Stomach & Skin Chicken & Veget...,Hill's Science Diet Adult Sensitive Stomach & ...,"Chicken Broth, Turkey, Chicken, Carrots, Potat...",NaN,Wet Food,27.9 %,19.6 %,...,NaN,NaN,121 ppm,NaN,703 IU/kg,1.21 %,5.43 %,NaN,NaN,NaN
3737,yorkshire terrier,congenital hypotrichosis,Inherited skin disorders,Adult Sensitive Stomach & Skin Salmon & Vegeta...,Hill's Science Diet Adult Sensitive Stomach & ...,"Chicken Broth, Salmon, Carrots, Turkey, Potato...",NaN,Wet Food,24.5 %,18.2 %,...,NaN,NaN,413 ppm,NaN,523 IU/kg,0.97 %,3.96 %,NaN,NaN,NaN
3738,yorkshire terrier,dermoid sinus,Inherited skin disorders,Adult Sensitive Stomach & Skin Tender Chicken ...,Hill's Science Diet Adult Sensitive Stomach & ...,"Chicken Broth, Chicken, Pork Liver, Carrots, R...",NaN,Wet Food,28.7 %,18.6 %,...,NaN,NaN,156 ppm,NaN,718 IU/kg,0.77 %,3.87 %,NaN,NaN,NaN
3739,yorkshire terrier,dermoid sinus,Inherited skin disorders,Adult Sensitive Stomach & Skin Chicken & Veget...,Hill's Science Diet Adult Sensitive Stomach & ...,"Chicken Broth, Turkey, Chicken, Carrots, Potat...",NaN,Wet Food,27.9 %,19.6 %,...,NaN,NaN,121 ppm,NaN,703 IU/kg,1.21 %,5.43 %,NaN,NaN,NaN
